### Setting up the data

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

In [ ]:
%%sql

CREATE DATABASE IF NOT EXISTS nyc

In [ ]:
%%sql

DROP TABLE IF EXISTS nyc.taxis

In [ ]:
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2021-04.parquet")
df.write.saveAsTable("nyc.taxis")

In [ ]:
%%sql -- Showcasing table
SELECT *
FROM nyc.taxis
LIMIT 5

In [ ]:
%%sql

SELECT COUNT(*) AS count
FROM nyc.taxis

### Schema Evolution

#### Changing names

In [ ]:
%%sql -- Changing names

ALTER TABLE nyc.taxis RENAME COLUMN fare_amount TO fare

In [ ]:
%%sql -- Changing names

ALTER TABLE nyc.taxis RENAME COLUMN trip_distance TO distance

#### Adjusting distance-column

In [ ]:
%%sql

ALTER TABLE nyc.taxis ALTER COLUMN distance COMMENT 'The elapsed trip distance in miles reported by the taximeter.'

In [ ]:
%%sql

ALTER TABLE nyc.taxis ALTER COLUMN distance TYPE double

In [ ]:
%%sql

ALTER TABLE nyc.taxis ALTER COLUMN distance AFTER fare

In [ ]:
%%sql -- Showcasing table
SELECT *
FROM nyc.taxis
LIMIT 5

#### Adding and updating a column

In [ ]:
%%sql

ALTER TABLE nyc.taxis ADD COLUMN  fare_per_distance_unit float AFTER distance

In [ ]:
%%sql -- Second Snapshot started

UPDATE nyc.taxis
SET fare_per_distance_unit = fare/distance

In [ ]:
%%sql

DESCRIBE EXTENDED nyc.taxis

#### Deleting column

In [ ]:
%%sql -- Third Snapshot started

DELETE FROM nyc.taxis
WHERE fare_per_distance_unit > 4.0 OR distance > 2.0

In [ ]:
%%sql -- Fourth Snapshot started

DELETE FROM nyc.taxis
WHERE fare_per_distance_unit is null

In [ ]:
%%sql 

SELECT COUNT(*) as count
FROM nyc.taxis

### Time Travel

In [ ]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM nyc.taxis

In [ ]:
%%sql

SELECT *
FROM nyc.taxis.history

In [ ]:
original_snapshot = ID
spark.sql(f"CALL system.rollback_to_snapshot('nyc.taxis', {original_snapshot})")

In [ ]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM nyc.taxis

In [ ]:
%%sql

SELECT *
FROM nyc.taxis.history

In [ ]:
%%sql

SELECT COUNT(*) as count
FROM nyc.taxis